# Main Codes used to obtain the plots of Paper 2

In [ ]:
import pandas as pd
import sys
sys.path.append("..")
from src.profit import *
import pandas as pd 
from src.genetic2 import Genetic
from influxdb import InfluxDBClient
from datetime import datetime
from dateutil import relativedelta
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from src.arimaPaper import Arima

Retrieve the production limits from the operator

In [ ]:
data = pd.read_csv('../data/Generator.csv')

limits = {}
for i in range(data.shape[0]):
    op = data.iloc[i]['operator']
    op = op.split("\'")
    try:
        if ',' in op[2]:
            if 'Bilateralista' not in op[3] and 'GAS RIMINI SPA' not in op[3]:
                if op[3].upper() not in limits:
                    limits[op[3].upper()] = 0
                limits[op[3].upper()] += data.iloc[i]['max_quantity']
    except:
        pass
    if '"]' in op[1]:
        op[1] = op[1].replace('"]', '')
    if 'Bilateralista' not in op[1] and 'GAS RIMINI SPA' not in op[1]:
        if op[1].upper() not in limits:
            limits[op[1].upper()] = 0
        limits[op[1].upper()] += data.iloc[i]['max_quantity']
        
limits['GAS RIMINI SPA'] = 0
for i in range(data.shape[0]):
    if 'GAS RIMINI SPA' in data.iloc[i]['operator']:
        limits['GAS RIMINI SPA'] += data.iloc[i]['max_quantity']

data = pd.read_csv('../data/predictedOps.csv').drop(columns=['Unnamed: 0']).set_index('op')

forecasted = []
for i in data.index:
    forecasted.append(i)

drop = []
for op in limits:
    if op not in forecasted:
        drop.append(op)
    else:
        forecasted.remove(op)

for op in drop:
    temp = op.split(' ')
    for di in forecasted:
        if temp[0] in di:
            inp = input(f'{op}->{di}')
            if inp == 'y':
                limits[op]
                limits[di] = limits[op]
                del limits[op]   

ops = []
vals = []
for op in limits:
    ops.append(op)
    vals.append(limits[op])
to_df = {
    'op':ops,
    'max':vals
}
pd.DataFrame(to_df).set_index('op').to_csv('../data/gencos.csv')

Predict the original value

In [ ]:
START = '2020/03/04'
OP_LIST = []

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'PublicBids')

for market in ['MGP', 'MI', 'MSD']:
    res = client.query(f"SELECT * FROM demand{market} WHERE time >= {START}").raw
    for val in res['series'][0]['values']:
        if val[3] not in OP_LIST and "'" not in val[3]:
            OP_LIST.append(val[3])

    res = client.query(f"SELECT * FROM supply{market} WHERE time >= {START}").raw
    for val in res['series'][0]['values']:
        if val[3] not in OP_LIST and "'" not in val[3]:
            OP_LIST.append(val[3])
pred = {
    'op':[],
    'MGPpO':[],
    'MGPqO':[],
    'MGPpD':[],
    'MGPqD':[],
    'MIpO':[],
    'MIqO':[],
    'MIpD':[],
    'MIqD':[],
    'MSDpO':[],
    'MSDqO':[],
    'MSDpD':[],
    'MSDqD':[]
}

for op in OP_LIST:
    prediction = Arima(op, TODAY, client).predict()
    pred['op'].append(op)
    pred['MGPpO'].append(prediction[0])
    pred['MGPqO'].append(prediction[1])
    pred['MGPpD'].append(prediction[2])
    pred['MGPqD'].append(prediction[3])
    pred['MIpO'].append(prediction[4])
    pred['MIqO'].append(prediction[5])
    pred['MIpD'].append(prediction[6])
    pred['MIqD'].append(prediction[7])
    pred['MSDpO'].append(prediction[8])
    pred['MSDqO'].append(prediction[9])
    pred['MSDpD'].append(prediction[10])
    pred['MSDqD'].append(prediction[11])

Find the smallest and the greatest operators

In [ ]:
data = pd.read_csv('../data/paperPred.csv').set_index('op')
limits = pd.read_csv('../data/gencos.csv').set_index('op')

min_val = 99999999
max_val = 0

for op in limits.index:
    if limits.loc[op]['max']<=min_val:
        min_val = limits.loc[op]['max']
        min_op = op
    if limits.loc[op]['max']>=max_val and limits.loc[op]['max']<191256490.77:
        max_val = limits.loc[op]['max']
        max_op = op
    if limits.loc[op]['max']>=191256490.77:
        limits = limits.drop(op)

print('Maximum')
print(max_op, max_val)
print('Minimum')
print(min_op, min_val)

# Case Study: Smallest Operator

In [ ]:
ga = Genetic(min_op, data, '2020-04-05T00:00:00Z', min_val)
p, s, _ = ga.run()

## $\Delta$ Profits

In [ ]:
p1 = getProfit(data, min_op, '2020-04-05T00:00:00Z')
p1

In [ ]:
p2 = getNewProfit(data, min_op, '2020-04-05T00:00:00Z', s[0][0])
p2

In [ ]:
d_mgp_o = p1[0]/p1[3]*100
d_mgp_n = p2[0]/p2[3]*100
d_mi_o = p1[1]/p1[3]*100
d_mi_n = p2[1]/p2[3]*100
d_msd_o = p1[2]/p1[3]*100
d_msd_n = p2[2]/p2[3]*100

print('Profit Percentage Change')
print('MGP')
print(round(d_mgp_o,2))
print(round(d_mgp_n,2))
print('MI')
print(round(d_mi_o,2))
print(round(d_mi_n,2))
print('MSD')
print(round(d_msd_o,2))
print(round(d_msd_n,2))

## $\Delta$ Resources Allocation

In [ ]:
data_f = data.loc[min_op]
sol = s[0][0]
delta = []
delta.append(sol[0]-data_f['MGPpO'])
delta.append((sol[1]-data_f['MGPqO']))
delta.append(sol[2]-data_f['MGPpD'])
delta.append((sol[3]-data_f['MGPqD']))
delta.append(sol[4]-data_f['MIpO'])
delta.append((sol[5]-data_f['MIqO']))
delta.append(sol[6]-data_f['MIpD'])
delta.append((sol[7]-data_f['MIqD']))
delta.append(sol[8]-data_f['MSDpO'])
delta.append((sol[9]-data_f['MSDqO']))
delta.append(sol[10]-data_f['MSDpD'])
delta.append((sol[11]-data_f['MSDqD']))
sol_o = [
    data_f['MGPpO'], data_f['MGPqO'], data_f['MGPpD'], 
    data_f['MGPqD'], data_f['MIpO'], data_f['MIqO'],
    data_f['MIpD'], data_f['MIqD'], data_f['MSDpO'],
    data_f['MSDqO'], data_f['MSDpD'], data_f['MSDqD']
]

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 13})
fig, ax = plt.subplots(figsize=(6.4, 4.8))
x = np.arange(len(delta))

bar_width = 0.4
ax2 = ax.twinx() 

cnt = 0
for i in np.arange(0,len(delta), 2):
    ax.bar(x[i]+cnt, delta[i],
        width=.8, color='steelblue')

    ax2.bar(x[i+1]+cnt, delta[i+1],
        width=.8, color='brown')

    cnt+=2.5

p = []
q = []
for i in range(len(delta)):
    if i%2 == 0:
        p.append(delta[i])
    else:
        q.append(delta[i])
        
plt.grid(linestyle='-.')    
tick = np.arange(0.5,25,4.5)  
plt.xticks(tick)
ax2.set_ylim(0, max(q)+1)
ax.set_ylim(0, max(p)+1)
ax.set_ylabel('$\Delta$ Bidded Prices [\u20ac/MWh]')
ax2.set_ylabel('$\Delta$ Bidded Quantities [MWh]')
plt.xticks(tick, labels=[
    'MGP\nOFF', 'MGP\nDEM', 'MI\nOFF', 'MI\nDEM', 'MSD\nOFF', 'MSD\nDEM'])    
ax.legend(['Prices'],loc='lower left', bbox_to_anchor= (.0, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
ax2.legend(['Quantities'],loc='lower left', bbox_to_anchor= (.3, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
plt.savefig('smallest_resource.png', transparent=True)

# Case Study: Greatest Operator

In [ ]:
ga = Genetic(max_op, data, '2020-04-05T00:00:00Z', max_val)
p, s, _= ga.run()

## $\Delta$ Profits

In [ ]:
p1 = getProfit(data, max_op, '2020-04-05T00:00:00Z')
p1

In [ ]:
p2 = getNewProfit(data, max_op, '2020-04-05T00:00:00Z', s[0][0])
p2

In [ ]:
d_mgp_o = p1[0]/p1[3]*100
d_mgp_n = p2[0]/p2[3]*100
d_mi_o = p1[1]/p1[3]*100
d_mi_n = p2[1]/p2[3]*100
d_msd_o = p1[2]/p1[3]*100
d_msd_n = p2[2]/p2[3]*100

print('Profit Percentage Change')
print('MGP')
print(round(d_mgp_o,2))
print(round(d_mgp_n,2))
print('MI')
print(round(d_mi_o,2))
print(round(d_mi_n,2))
print('MSD')
print(round(d_msd_o,2))
print(round(d_msd_n,2))

## $\Delta$ Resources Allocation

In [ ]:
data_f = data.loc[max_op]
sol = s[0][0]
delta = []
delta.append(sol[0]-data_f['MGPpO'])
delta.append((sol[1]-data_f['MGPqO']))
delta.append(sol[2]-data_f['MGPpD'])
delta.append((sol[3]-data_f['MGPqD']))
delta.append(sol[4]-data_f['MIpO'])
delta.append((sol[5]-data_f['MIqO']))
delta.append(sol[6]-data_f['MIpD'])
delta.append((sol[7]-data_f['MIqD']))
delta.append(sol[8]-data_f['MSDpO'])
delta.append((sol[9]-data_f['MSDqO']))
delta.append(sol[10]-data_f['MSDpD'])
delta.append((sol[11]-data_f['MSDqD']))
sol_o = [
    data_f['MGPpO'], data_f['MGPqO'], data_f['MGPpD'], 
    data_f['MGPqD'], data_f['MIpO'], data_f['MIqO'],
    data_f['MIpD'], data_f['MIqD'], data_f['MSDpO'],
    data_f['MSDqO'], data_f['MSDpD'], data_f['MSDqD']
]

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 13})
fig, ax = plt.subplots(figsize=(6.4, 4.8))
x = np.arange(len(delta))

bar_width = 0.4
ax2 = ax.twinx() 

cnt = 0
for i in np.arange(0,len(delta), 2):
    ax.bar(x[i]+cnt, delta[i],
        width=.8, color='steelblue')

    ax2.bar(x[i+1]+cnt, delta[i+1],
        width=.8, color='brown')

    cnt+=2.5

p = []
q = []
for i in range(len(delta)):
    if i%2 == 0:
        p.append(delta[i])
    else:
        q.append(delta[i])
lim_p = max(abs(min(p)), abs(max(p)))
lim_q = max(abs(min(q)), abs(max(q)))
plt.grid(linestyle='-.')    
tick = np.arange(0.5,25,4.5)  
plt.xticks(tick)
ax2.set_ylim(-lim_q-1000, lim_q+1000)
ax.set_ylim(-lim_p-10, lim_p+10)
ax.set_ylabel('$\Delta$ Bidded Prices [\u20ac/MWh]')
ax2.set_ylabel('$\Delta$ Bidded Quantities [MWh]')
plt.xticks(tick, labels=[
    'MGP\nOFF', 'MGP\nDEM', 'MI\nOFF', 'MI\nDEM', 'MSD\nOFF', 'MSD\nDEM'])    
ax.legend(['Prices'],loc='lower left', bbox_to_anchor= (.0, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
ax2.legend(['Quantities'],loc='lower left', bbox_to_anchor= (.3, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
plt.savefig('greatest_resource.png', transparent=True)

# Case Study: Iren for paper 1

In [ ]:
ga = Genetic('IREN ENERGIA SPA', data, '2020-04-05T00:00:00Z', 116543.59999999999)
p, s, _= ga.run()

## $\Delta$ Profits

In [ ]:
p1 = getProfit(data, 'IREN ENERGIA SPA', '2020-04-05T00:00:00Z')
p1

In [ ]:
p2 = getNewProfit(data, 'IREN ENERGIA SPA', '2020-04-05T00:00:00Z', s[0][0])
p2

In [ ]:
if p1[0] == 0.0:
    d_mgp = (p2[0] - p1[0])/1*100
else:
    d_mgp = (p2[0] - p1[0])/abs(p1[0])*100
if p1[1] == 0.0:
    d_mi = (p2[1] - p1[1])/1*100
else:
    d_mi = (p2[1] - p1[1])/abs(p1[1])*100
if p1[2] == 0.0:
    d_msd = (p2[2] - p1[2])/1*100
else:
    d_msd = (p2[2] - p1[2])/abs(p1[2])*100
if p1[3] == 0.0:
    d_tot = (p2[3] - p1[3])/1*100
else:
    d_tot = (p2[3] - p1[3])/abs(p1[3])*100

print('Profit Percentage Change')
print(f'MGP: {round(d_mgp,2)}%')
print(f'MI:  {round(d_mi,2)}%')
print(f'MSD: {round(d_msd,2)}%')
print(f'Tot: {round(d_tot,2)}%')

## $\Delta$ Resources Allocation

In [ ]:
data_f = data.loc[max_op]
sol = s[0][0]
delta = []
delta.append(sol[0]-data_f['MGPpO'])
delta.append((sol[1]-data_f['MGPqO']))
delta.append(sol[2]-data_f['MGPpD'])
delta.append((sol[3]-data_f['MGPqD']))
delta.append(sol[4]-data_f['MIpO'])
delta.append((sol[5]-data_f['MIqO']))
delta.append(sol[6]-data_f['MIpD'])
delta.append((sol[7]-data_f['MIqD']))
delta.append(sol[8]-data_f['MSDpO'])
delta.append((sol[9]-data_f['MSDqO']))
delta.append(sol[10]-data_f['MSDpD'])
delta.append((sol[11]-data_f['MSDqD']))
sol_o = [
    data_f['MGPpO'], data_f['MGPqO'], data_f['MGPpD'], 
    data_f['MGPqD'], data_f['MIpO'], data_f['MIqO'],
    data_f['MIpD'], data_f['MIqD'], data_f['MSDpO'],
    data_f['MSDqO'], data_f['MSDpD'], data_f['MSDqD']
]

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 13})
fig, ax = plt.subplots(figsize=(6.4, 4.8))
x = np.arange(len(delta))

bar_width = 0.4
ax2 = ax.twinx() 

cnt = 0
for i in np.arange(0,len(delta), 2):
    ax.bar(x[i]+cnt, delta[i],
        width=.8, color='steelblue')

    ax2.bar(x[i+1]+cnt, delta[i+1],
        width=.8, color='brown')

    cnt+=2.5

p = []
q = []
for i in range(len(delta)):
    if i%2 == 0:
        p.append(delta[i])
    else:
        q.append(delta[i])
lim_p = max(abs(min(p)), abs(max(p)))
lim_q = max(abs(min(q)), abs(max(q)))
plt.grid(linestyle='-.')    
tick = np.arange(0.5,25,4.5)  
plt.xticks(tick)
ax2.set_ylim(-lim_q-1000, lim_q+1000)
ax.set_ylim(-lim_p-10, lim_p+10)
ax.set_ylabel('$\Delta$ Bidded Prices [\u20ac/MWh]')
ax2.set_ylabel('$\Delta$ Bidded Quantities [MWh]')
plt.xticks(tick, labels=[
    'MGP\nOFF', 'MGP\nDEM', 'MI\nOFF', 'MI\nDEM', 'MSD\nOFF', 'MSD\nDEM'])    
ax.legend(['Prices'],loc='lower left', bbox_to_anchor= (.0, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
ax2.legend(['Quantities'],loc='lower left', bbox_to_anchor= (.3, 1.01), ncol=2,
            borderaxespad=0, frameon=False)

# Case Study: All the Operators

In [ ]:
profs = []
sols = []
ops = []
for op in limits.index:
    try:
        ga = Genetic(op, data, '2020-04-05T00:00:00Z', limits.loc[op]['max'])
        p, s, _= ga.run()
        profs.append(p)
        sols.append(s)
        ops.append(op)
    except:
        pass

In [ ]:
new_sol = []
for i in sols:
    new_sol.append(sols[0][0][0])
to_df = {
    'op':ops,
    'sol':new_sol
}
df = pd.DataFrame(to_df).set_index('op')
df.to_csv('../data/analysis.csv')

In [ ]:
old_p = []
new_p = []
for i in df.index:
    old_p.append(getProfit(data, i, '2020-04-05T00:00:00Z'))
    new_p.append(getNewProfit(data, i, '2020-04-05T00:00:00Z', df.loc[i]['sol']))

In [ ]:
old_m = []
new_m = []
mgp_old_m = []
mgp_new_m = []
mi_old_m = []
mi_new_m = []
msd_old_m = []
msd_new_m = []
for i in range(len(old_p)):
    if old_p[i][3] != 0.0:
        mgp_old = old_p[i][0]/old_p[i][3]*100
        mgp_new = new_p[i][0]/new_p[i][3]*100
        mgp_old_m.append(mgp_old)
        mgp_new_m.append(mgp_new)
        
        mi_old = old_p[i][1]/old_p[i][3]*100
        mi_new = new_p[i][1]/new_p[i][3]*100
        mi_old_m.append(mi_old)
        mi_new_m.append(mi_new)
    
        msd_old = old_p[i][2]/old_p[i][3]*100
        msd_new = new_p[i][2]/new_p[i][3]*100
        msd_old_m.append(msd_old)
        msd_new_m.append(msd_new)
        
        old_m.append(old_p[i][3])
        new_m.append(new_p[i][3])
        
print('MGP')
print(round(np.mean(mgp_old_m), 2))      
print(round(np.mean(mgp_new_m), 2))

print('MI')
print(round(np.mean(mi_old_m), 2))      
print(round(np.mean(mi_new_m), 2))

print('MSD')
print(round(np.mean(msd_old_m), 2))      
print(round(np.mean(msd_new_m), 2))

print(np.mean(old_m))
print(np.mean(new_m))

In [ ]:
temp_delta = [
    [],[],[],[],[],[],[],[],[],[],[],[]
]
for op in df.index:
    data_f = data.loc[op]
    sol = df.loc[op]['sol']
    temp_delta[0].append(sol[0]-data_f['MGPpO'])
    temp_delta[1].append((sol[1]-data_f['MGPqO']))
    temp_delta[2].append(sol[2]-data_f['MGPpD'])
    temp_delta[3].append((sol[3]-data_f['MGPqD']))
    temp_delta[4].append(sol[4]-data_f['MIpO'])
    temp_delta[5].append((sol[5]-data_f['MIqO']))
    temp_delta[6].append(sol[6]-data_f['MIpD'])
    temp_delta[7].append((sol[7]-data_f['MIqD']))
    temp_delta[8].append(sol[8]-data_f['MSDpO'])
    temp_delta[9].append((sol[9]-data_f['MSDqO']))
    temp_delta[10].append(sol[10]-data_f['MSDpD'])
    temp_delta[11].append((sol[11]-data_f['MSDqD']))

delta = []
for i in range(len(temp_delta)):
    delta.append(np.mean(temp_delta[i]))

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 13})
fig, ax = plt.subplots(figsize=(6.4, 4.8))
x = np.arange(len(delta))

bar_width = 0.4
ax2 = ax.twinx() 

cnt = 0
for i in np.arange(0,len(delta), 2):
    ax.bar(x[i]+cnt, delta[i],
        width=.8, color='steelblue')

    ax2.bar(x[i+1]+cnt, delta[i+1],
        width=.8, color='brown')

    cnt+=2.5

p = []
q = []
for i in range(len(delta)):
    if i%2 == 0:
        p.append(delta[i])
    else:
        q.append(delta[i])
lim_p = max(abs(min(p)), abs(max(p)))
lim_q = max(abs(min(q)), abs(max(q)))
plt.grid(linestyle='-.')    
tick = np.arange(0.5,25,4.5)  
plt.xticks(tick)
ax2.set_ylim(-lim_q-1000, lim_q+1000)
ax.set_ylim(-lim_p-10, lim_p+10)
ax.set_ylabel('$\Delta$ Bidded Prices [\u20ac/MWh]')
ax2.set_ylabel('$\Delta$ Bidded Quantities [MWh]')
plt.xticks(tick, labels=[
    'MGP\nOFF', 'MGP\nDEM', 'MI\nOFF', 'MI\nDEM', 'MSD\nOFF', 'MSD\nDEM'])    
ax.legend(['Prices'],loc='lower left', bbox_to_anchor= (.0, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
ax2.legend(['Quantities'],loc='lower left', bbox_to_anchor= (.3, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
plt.savefig('../fig/avg_resource.png', transparent=True)